### Importing packages

In [1]:
import os
import csv
import sys
import nltk
import spacy
import pandas as pd
import hashlib
from pandas.io.common import EmptyDataError
from pathlib import Path

### Counting Total number of words in the reddit data set

In [2]:
os.chdir("C:/Users/ssund/PycharmProjects/adm_project1/reddit")

word_count = 0
for fileName in Path('.').glob('*.txt'):
    with open(str(fileName.absolute()), 'r', encoding='utf-8') as f:
        for line in f:
            word = line.split()
            word_count += len(word)
print("Total number of words in the reddit dataset is", word_count)

Total number of words in the reddit dataset is 54987183


### Extracting WIKI words from the data set

In [3]:
wiki1="C:/Users/ssund/PycharmProjects/adm_project1/wiki_word_dataset.txt"
f1= open(wiki1,"w")
for fileName in Path('.').glob('*.txt'):  
    lines = ""
    wikiword = ""
    with open(str(fileName.absolute()),'r',encoding="utf8") as one_text:
        lines=one_text.read()
        splitwords=lines.split(" ")    
        for i in splitwords:
            if (i.lower()).startswith("wiki_"):       
                wikiword=wikiword+i+"\n"
        f1.write(wikiword)
f1.close()
print("wiki_word_dataset has been created")

wiki_word_dataset has been created


### Counting the total number of Wiki Words

In [4]:
wiki_data = "C:/Users/ssund/PycharmProjects/adm_project1/wiki_word_dataset.txt"
count = len(open(wiki_data).readlines())
print("Total number of wiki words in the dataset:", count)

os.chdir('C:/Users/ssund/PycharmProjects/adm_project1')

Total number of wiki words in the dataset: 1321967


### Finding the frequency of each Wiki word

In [5]:
with open ("wiki_word_dataset.txt", "r") as myfile:
    data=myfile.read().replace('\n', ' ')

data = data.split(' ')
fdist1 = nltk.FreqDist(data)
sys.stdout = open("frequency.txt","w")
print(fdist1.most_common())

### Removing the Wikifier add-ons at the beginning and at the end

In [6]:
f1=open("wiki_word_dataset.txt",'r')
f2=open("wiki_clean.txt",'w')

completed_lines_hash = set()

for w in f1:
    f2.write(w.rpartition("__")[0].replace("__", ".").replace("Wiki.","").replace("."," ").replace("_"," ").lower() + "\n")
f1.close()
f2.close()

### Frequency of every word after removing the Wikifier add-ons at the beginning and at the end

In [7]:
with open ("wiki_clean.txt", "r") as myfile1:
    data1=myfile1.read().replace('\n', ' ')

data1 = data1.split(' ')    
fdist2 = nltk.FreqDist(data1)
sys.stdout=open("wikiwords_frequency.txt","w")
print(fdist2.most_common())

### Removing the duplicate words in the file before comparing with BERT Vocab

In [8]:
output_file = open("OUTPUT.txt", "w")
f=open("wiki_clean.txt",'r')
for line in f:
  hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()
  if hashValue not in completed_lines_hash:
    output_file.write(line)
    completed_lines_hash.add(hashValue)

### Matching the WikiWords with BERT vocabulary and printing them to a new text file

In [9]:
file3 = open('Matching_with_BERT.txt','w')
file1 = set(line.strip() for line in open('OUTPUT.txt', 'r', encoding="utf8"))
file2 = set(line.strip() for line in open('BERT_Vocabulary.txt', 'r', encoding="utf8"))

for line in file1 & file2:

    if line:
        file3.write(line + "\n")
        
file3.close()

### List 1: only the wiki words that contain a one single word

In [10]:
a = open("OUTPUT.txt",'r')
b = open("single_words.txt",'w')

for i in a:
    for j in i:
        if j==" ":
            break
    if j!=" ":
        b.write(i)

### List 2: printing everything else into different lines because Bert expects only individual words

In [11]:
d = open('OUTPUT.txt','r')
c = open("multiple_words.txt",'w')

for i in d:
    for j in i.split():
        c.write(j)
        c.write('\n')

### Matching both the lists to BERT Vocab and printing the words which are there in BERT Vocab to a new text file

In [ ]:
e = set(line.strip() for line in open('single_words.txt', 'r', encoding="utf8"))
match_single = open('Final_Matching_Single_Words_with_BERT.txt','w')


for line in e & file2:

    if line:
        match_single.write(line + "\n")


f = set(line.strip() for line in open('multiple_words.txt', 'r', encoding="utf8"))
match_multiple = open('Final_Matching_Multiple_Words_with_BERT.txt','w')


for line in f & file2:

    if line:
        match_multiple.write(line + "\n")


file3.close()        
sys.stdout.close()
a.close()
b.close()
c.close()
d.close()
match_single.close()

### Analysis of the WIKI words in the dataset

In [ ]:
nlp = spacy.load('en_core_web_sm') 

f1 = open('Final_Matching_Multiple_Words_with_BERT.txt','r')
f2 = open('analysis_multiple_word.txt', 'w')  
doc = nlp(f1.read()) 

for ent in doc.ents: 
    op1 = ent.text + '\n'
    op2=ent.label_ + ' '
    f2.write(op2)
    f2.write(op1)
    a=sorted(f2)  
    for i in a:
         s += i
    f2.write(s)
f1.close()
f2.close()

In [ ]:
# The Analysis is done based on the below table from spacy library:

# TYPE            DESCRIPTION
# PERSON          People, including fictional.
# NORP            Nationalities or religious or political groups.
# FAC             Buildings, airports, highways, bridges, etc.
# ORG             Companies, agencies, institutions, etc.
# GPE             Countries, cities, states.
# LOC             Non-GPE locations, mountain ranges, bodies of water.
# PRODUCT         Objects, vehicles, foods, etc. (Not services.)
# EVENT           Named hurricanes, battles, wars, sports events, etc.
# WORK_OF_ART     Titles of books, songs, etc.
# LAW             Named documents made into laws.
# LANGUAGE        Any named language.
# DATE            Absolute or relative dates or periods.
# TIME            Times smaller than a day.
# PERCENT         Percentage, including ”%“.
# MONEY           Monetary values, including unit.
# QUANTITY        Measurements, as of weight or distance.
# ORDINAL         “first”, “second”, etc.
# CARDINAL        Numerals that do not fall under another type

# SORT the above data:

# f1=open('analysis_multiple_word.txt','r')
# f2=open("analysis_multiple_word_sort.txt",'w')
# a=sorted(f1)
# s=""
# for i in a:
#     s += i
# f2.write(s)
# f1.close()
# f2.close()

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "C:\Users\ssund\Anaconda3\lib\site-packages\IPython\core\history.py", line 780, in writeout_cache
    self._writeout_input_cache(conn)
  File "C:\Users\ssund\Anaconda3\lib\site-packages\IPython\core\history.py", line 764, in _writeout_input_cache
    (self.session_number,)+line)
sqlite3.IntegrityError: UNIQUE constraint failed: history.session, history.line

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ssund\Anaconda3\lib\site-packages\IPython\core\history.py", line 834, in run
    self.history_manager.writeout_cache(self.db)
  File "<C:\Users\ssund\Anaconda3\lib\site-packages\decorator.py:decorator-gen-23>", line 2, in writeout_cache
  File "C:\Users\ssund\Anaconda3\lib\site-packages\IPython\core\history.py", line 58, in needs_sqlite
    return f(self, *a, **kw)
  File "C:\Users\ssund\Anaconda3\lib\site-packages\IPython\c